# Change Log (v.4.5)
- More clear explantation
- sentence based data (5 dimensions: sentid, wid, mid, char, features)

# Import

In [1]:
%load_ext autoreload

In [2]:
%autoreload
# -*- coding: utf-8 -*-
'''
    An implementation of sequence to sequence learning
    for performing ensemble morphosyntactic analyses
'''
from __future__ import print_function
# from keras.preprocessing.sequence import pad_sequences
import numpy as np
from six.moves import range
from prepare_data import SawarefData, padIndexes
from character_table import colors, CharacterTable, eprint
import pandas as pd
import itertools
import re
from ws_client import WebSocketClient
import pickle
import sys
import datetime
from keras.callbacks import TensorBoard
from buckwalter import utf2bw, bw2utf
from pprint import pprint

# do not import in interactive mode
# from vis import SawarefVis
from keras.models import Sequential, Model, load_model
from keras import layers
from keras.callbacks import EarlyStopping, Callback
from keras.utils import plot_model

Using TensorFlow backend.


# Constants

## Experiemnts Names
### Compartative: 
- Morpheme-based (ru, sp1,),  `comp_ru`, `comp_sp1`, 
- character-based, `comp_ch`
- end-to-end `comp_end`, 


## Experiment Class and Initialization

`e.feat_x` is the input categorical features

`e.feat_y` is the output categorical features

`e.strings_x` is the input character-based strings features

`e.strings_y` is the output character-based strings features

In [13]:
class Experiment:
    feat_x = [
        "MXpos", "STpos", "AMpos", "FApos", 
        "STaspect", "AMaspect", "MXaspect", "FAaspect", 
        "STperson", "AMperson", "MXperson", "FAperson", 
        "STgender", "AMgender", "MXgender", "FAgender", 
        "STnumber", "AMnumber", "MXnumber", "FAnumber", 
        "STcase", "AMcase", "MXcase", "FAcase", 
        "STvoice", "AMvoice", "MXvoice", "FAvoice", 
        "STmood", "AMmood", "MXmood", "FAmood", 
        "STstate", "AMstate", "MXstate", "FAstate"
    ]
    feat_y = [
        "QApos", "QAaspect", "QAperson", "QAgender", "QAnumber", "QAcase",
        "QAvoice", "QAmood", "QAstate"
    ]
    strings_x = ['in_letter','in_diac']
    strings_y = ['out_letter','out_diac','seg']
    instrings_x = ["STwutf8"] # This is the source of string_x later on
    instrings_y = ["QAutf8"] # This is the source of string_x later on
    MYPATH = "/morpho/output/"
    # Parameters for the model and dataset.
    TRAINING_SIZE = 50000
    EPOCHS = 100
    EMBEDDINGS = 0
    # DIGITS = 3
    # REVERSE = True
    # Try replacing GRU, or SimpleRNN.
    HIDDEN_SIZE = 128
    BATCH_SIZE = 64
    LAYERS = 1
    ITERATIONS = 10
    REVERSE = False
    MODEL_NAME = "main-seq-multiinput-multioutput-segmentation.keras"
    DATA_PICKLE = "main-seq-multiinput-multioutput-segmentation.pickle"
    RNN = layers.LSTM
    CATS_EMBEDDING = 1000
    TEST_SPLIT = 1  #  means 0.1 of data is for test
    VAL_SPLIT = 1  #  means 0.1 of data is for test
    LOAD_FASTTEXT = False
    CHARACTER_BASED = False
    MORPHEME_BASED = False
    previous_names = set()
    SHUFFLE = True
    
    def __init__(self, exp_type="e"):
        self.exp_type = exp_type
        self.set_constants()
        self.set_name()
        self.load_emb()
    
    def load_emb(self):
        if self.LOAD_FASTTEXT:
            self.emb = WebSocketClient("ws://localhost:8765/")
    def set_name(self):
        self.NAME = self.exp_type+"_"+input("What is the name of this experiment? ")
        self.thedate = datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M")
        print("\nNAME=", self.NAME)
        print("DATE=", self.thedate)
        self.previous_names.add(self.NAME+"%"+self.thedate)
        self.writer = pd.ExcelWriter('results_'+self.NAME+'.xlsx')
    
    def set_constants(self):
        self.MYPATH = "/morpho/output/"
        # Parameters for the model and dataset.
        self.TRAINING_SIZE = 50000
        self.EPOCHS = 40
        self.EMBEDDINGS = 0
        # DIGITS = 3
        # REVERSE = True
        # Try replacing GRU, or SimpleRNN.
        self.HIDDEN_SIZE = 128
        self.BATCH_SIZE = 64
        self.LAYERS = 1
        self.ITERATIONS = 10
        self.MODEL_NAME = "main-seq-multiinput-multioutput-segmentation.keras"
        self.DATA_PICKLE = "main-seq-multiinput-multioutput-segmentation.pickle"
        self.RNN = layers.LSTM
        if self.exp_type == "comp_ru" or self.exp_type == "comp_sp1":
            self.CATS_EMBEDDING = 1000
            self.TEST_SPLIT = 1  #  means 0.1 of data is for test
            self.VAL_SPLIT = 1  #  means 0.1 of data is for test
            self.LOAD_FASTTEXT = True
            self.CHARACTER_BASED = False
            self.MORPHEME_BASED = True
            self.ALIGN_TYPE = "ru-" if self.exp_type == "comp_ru" else "sp1-"
        elif self.exp_type == "comp_ch":
            self.CATS_EMBEDDING = 1000
            self.TEST_SPLIT = 1  #  means 0.1 of data is for test
            self.VAL_SPLIT = 1  #  means 0.1 of data is for test
            self.LOAD_FASTTEXT = False
            self.CHARACTER_BASED = True
            self.MORPHEME_BASED = False
            self.ALIGN_TYPE = "ch-"            
        elif self.exp_type == "comp_ench":
            self.CATS_EMBEDDING = 1000
            self.TEST_SPLIT = 0  #  means 0.1 of data is for test
            self.VAL_SPLIT = 1  #  means 0.1 of data is for test
            self.LOAD_FASTTEXT = False
            self.CHARACTER_BASED = False
            self.MORPHEME_BASED = False
            self.ALIGN_TYPE = "no-"            
        elif self.exp_type.startswith("comp_end"):
            self.CATS_EMBEDDING = 1000
            self.TEST_SPLIT = 1  #  means 0.1 of data is for test
            self.VAL_SPLIT = 1  #  means 0.1 of data is for test
            self.LOAD_FASTTEXT = True
            self.CHARACTER_BASED = False
            self.MORPHEME_BASED = False
            self.ALIGN_TYPE = "no-"

            stripped = self.exp_type.lstrip("comp_end_")
            print(stripped)
            if stripped in ["MX", "ST", "FA", "AM"]:
                self.feat_x = [x for x in self.feat_x if x[0:2] == stripped]
            elif len(stripped.split("_")) >= 2:
                self.feat_x = [x for x in self.feat_x if x[0:2] in stripped.split("_")]
            elif stripped == "baseline":
                self.feat_x = []
        else:
            print("ERROR: Not a valid config name")
            raise NameError("ERROR: Not a valid config name")
        
        if self.exp_type.startswith("comp_end_stateful"):
            self.SHUFFLE = False

        if self.CHARACTER_BASED:
            self.strings_x = ["bw"]
        elif self.MORPHEME_BASED:
            self.strings_x = ["word"]

In [14]:
e = Experiment("comp_ench")

What is the name of this experiment? 4d

NAME= comp_ench_4d
DATE= .2018.09.14.16.00


In [15]:
e.LOAD_MODEL = True if input("Load Model? (y/N)") == "y" else False
print("Loading Model") if e.LOAD_MODEL else print("Training a new model. Old one with same name will be overwritten.")

Load Model? (y/N)
Training a new model. Old one with same name will be overwritten.


In [16]:
try:
    writer_all
except NameError:
    writer_all = pd.ExcelWriter('all_results.xlsx')

## Top-level variables (accuracies, writer_all)

In [17]:
try:
    accuracies
except NameError:
    accuracies = pd.DataFrame([],
                 columns=list(map(list, zip(*[
        ('QAaspect', 'acc'),('QAaspect', 'IOV_acc'),('QAaspect', 'OOV_acc'),
        ('QAcase', 'acc'),('QAcase', 'IOV_acc'),('QAcase', 'OOV_acc'),
        ('QAgender', 'acc'),('QAgender', 'IOV_acc'),('QAgender', 'OOV_acc'),
        ('QAmood', 'acc'),('QAmood', 'IOV_acc'),('QAmood', 'OOV_acc'),
        ('QAnumber', 'acc'),('QAnumber', 'IOV_acc'),('QAnumber', 'OOV_acc'),
        ('QAperson', 'acc'),('QAperson', 'IOV_acc'),('QAperson', 'OOV_acc'),
        ('QApos', 'acc'),('QApos', 'IOV_acc'),('QApos', 'OOV_acc'),
        ('QAupos', 'acc'),('QAupos', 'IOV_acc'),('QAupos', 'OOV_acc'),                     
        ('QAstate', 'acc'),('QAstate', 'IOV_acc'),('QAstate', 'OOV_acc'),
        ('QAvoice', 'acc'),('QAvoice', 'IOV_acc'),('QAvoice', 'OOV_acc'),
        ('QAutf8', 'acc'),('QAutf8', 'IOV_acc'),('QAutf8', 'OOV_acc'),
        ('agg', 'all'),
        ('agg', 'mf'),
        ('agg', 'utf8')]))))
try:
    h_accuracies
except NameError:
    h_accuracies = pd.DataFrame([],
                 columns=list(map(list, zip(*[
        ('QAaspect', 'acc'),('QAaspect', 'IOV_acc'),('QAaspect', 'OOV_acc'),
        ('QAcase', 'acc'),('QAcase', 'IOV_acc'),('QAcase', 'OOV_acc'),
        ('QAgender', 'acc'),('QAgender', 'IOV_acc'),('QAgender', 'OOV_acc'),
        ('QAmood', 'acc'),('QAmood', 'IOV_acc'),('QAmood', 'OOV_acc'),
        ('QAnumber', 'acc'),('QAnumber', 'IOV_acc'),('QAnumber', 'OOV_acc'),
        ('QAperson', 'acc'),('QAperson', 'IOV_acc'),('QAperson', 'OOV_acc'),
        ('QApos', 'acc'),('QApos', 'IOV_acc'),('QApos', 'OOV_acc'),
        ('QAupos', 'acc'),('QAupos', 'IOV_acc'),('QAupos', 'OOV_acc'),                     
        ('QAstate', 'acc'),('QAstate', 'IOV_acc'),('QAstate', 'OOV_acc'),
        ('QAvoice', 'acc'),('QAvoice', 'IOV_acc'),('QAvoice', 'OOV_acc'),
        ('QAutf8', 'acc'),('QAutf8', 'IOV_acc'),('QAutf8', 'OOV_acc'),
        ('agg', 'all'),
        ('agg', 'mf'),
        ('agg', 'utf8')]))))

## Functions

In [168]:
def padIndexes(df, axis):
    prev = df.index[0]
    limit = max([df.index.get_level_values(axis) ] ) 
    sett = list(df.index)
    for x in df.index:
        # print(df.index)
        if x[:-1] != prev[:-1]:
            for p in range(prev[-1] + 1, limit + 1):
                sett.append(prev[:-1] + (p,))
        prev = x
    for p in range(prev[-1] + 1, limit + 1):
        sett.append(prev[:-1] + (p,))
    return sett

def pretty_join(arr):
    if isinstance(arr, pd.Series):
        arr = arr.to_frame().T
    if isinstance(arr.columns, pd.core.index.MultiIndex):
        return "/".join([
            '+'.join([
                x[1] for x in arr.columns[row == 1]
                if x[1][-2:] != "na" and x[1][-2:] != "_0"
            ]) for index, row in arr.iterrows()
        ])
    else:
        return "/".join([
            '+'.join([
                x for x in arr.columns[row == 1]
                if x[-2:] != "na" and x[-2:] != "_0"
            ]) for index, row in arr.iterrows()
        ])


def pretty_value(colum_value):
    return re.sub(".*_", "", colum_value)


def getValuesAndReshape(df, middle_dim):
    return df.values.reshape((df.shape[0] // middle_dim, middle_dim, -1))


#     return df.values.reshape((df.shape[0]//middle_dim, middle_dim, df.shape[1]))


def flattencolumns(df1, cols):
    df = pd.concat(
        [pd.DataFrame(df1[x].values.tolist()).add_prefix(x) for x in cols],
        axis=1)
    return pd.concat([df, df1.drop(cols, axis=1)], axis=1)


def truncate(x):
    return x[:EMBEDDINGS]


def removeDiac(x):
    return re.sub('[ًٌٍَُِّْ��]', '', x.replace("ٱ","ا"))


def padStringWithSpaces(x):
    return x + ' ' * (e.STRING_LENGTH - len(x))


def joinMorphemesStrings(arr):
    return "+".join([
        str(x) for x in arr if isinstance(x, float) == False and x != -1
        and x != "-----" and x != "-"
    ])


def fullprint(*args, **kwargs):
    opt = np.get_printoptions()
    np.set_printoptions(threshold='nan')
    pprint(*args, **kwargs)
    np.set_printoptions(**opt)
    
    
def pretty_join2(arr, columns, v):
    return "/".join([columns[row].replace(v+"_","") for row in arr]).rstrip("/0")    


class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        metrics = self.model.evaluate(x, y, batch_size=e.BATCH_SIZE, verbose=0)
        for i,x in enumerate(self.model.metrics_names):
            logs["test_"+x]= metrics[i]
        logs['test_acc']= np.mean([logs["test_"+x] for x in self.model.metrics_names if x[-4:]=='_acc'])
        logs['val_acc']= np.mean([logs["val_"+x] for x in self.model.metrics_names if x[-4:]=='_acc'])
        logs['train_acc']= np.mean([logs[x] for x in self.model.metrics_names if x[-4:]=='_acc'])
        return logs

# Load and Preprocess Data

## Loading data from sawaref

In [116]:
try:
    results_all
except NameError:
    results_all = {}
if e.NAME in results_all:
    print("Warning: same name exits. Results will be overwritten")
else:
    results_all[e.NAME] = {}

In [172]:
%autoreload
from prepare_data import SawarefData
sawarefData = SawarefData(
    e.MYPATH,
    e.EMBEDDINGS,
    align_type=e.ALIGN_TYPE,
    feat_x=e.feat_x,
    strings_x=e.instrings_x,
    strings_y=e.instrings_y,
    feat_y=e.feat_y)

Reading from file: morpho.output.no-.pickle
1098
Vectorization...


## DF dataframe (the origin of everthing)

In [192]:
source = list(itertools.chain(*sawarefData.quran_sent))
df = pd.DataFrame(
    source,
    columns=["sid", "aid", "wid", "mid"] + e.feat_x + 
    e.instrings_x + e.instrings_y+
    ["embeddings"] + e.feat_y)
if e.EMBEDDINGS > 0:
    df["embeddings"] = df["embeddings"].apply(truncate)
    df = flattencolumns(df, ["embeddings"])
df.set_index(["sid", "aid", "wid", "mid"], inplace=True)
df.sort_index(inplace=True)

### Some Santizing

In [193]:
df.replace("0", 0, inplace=True)

### a. clean all padded rows

In [194]:
strings = e.instrings_x + e.instrings_y

for s in strings:
    df.loc[pd.isna(df[s]), s] = ""

### 2. Pad the rows according to the longest word (in # of morphemes)

In [195]:
e.SENTLEN = max(df.index.get_level_values("mid"))
df = df.reindex(
    padIndexes(df, 3),
    fill_value=-1).sort_index()

TypeError: 'Int64Index' object cannot be interpreted as an integer

In [177]:
for x in e.feat_y:
    df.loc[df["QAutf8"]=="",x] = -1
for x in e.strings_y:
    df.loc[df["QAutf8"]=="",x] = -1
    df.loc[df["QAutf8"]=="",x+"_undiac"] = -1

In [185]:
## 3. Get the hot encoding of all caterogirical data (see columns attr)
dumm = pd.get_dummies(df, columns=e.feat_x + e.feat_y)

In [186]:
for x in e.feat_x:
    dumm.drop(x+"_-1",axis=1,inplace=True)

In [187]:
dumm.drop("embeddings", axis=1, inplace=True)

In [183]:
dumm.index = [[x for x in range(e.EXAMPLES_LEN) for _ in range(e.SENTLEN)],
              [x for _ in range(e.EXAMPLES_LEN) for x in range(e.SENTLEN)]]


ValueError: Length mismatch: Expected axis has 50741 elements, new values have 50740 elements

In [184]:
## 4. Add two-level columns for easy indexing later (wid, mid)
e.EXAMPLES_LEN = df.shape[0] // e.SENTLEN
new_columns = []
for x in dumm.columns:
    new_columns.append(re.sub('(_.*|[0-9]*)', '', x))
dumm.columns = [new_columns, dumm.columns]
dumm.sort_index(axis=1, inplace=True)

TypeError: expected string or bytes-like object

In [182]:
## 5. Find possible values of each cat
def getSet(df):
    results = set()
    df.apply(results.add)
    return results


embeddingInputSets = {i: getSet(df[i]) for i in e.feat_x}

In [152]:
e.feat_x = list(set(e.feat_x) - set([x for x in e.feat_x if len( embeddingInputSets[x])<=1]))

In [153]:
df2 = pd.concat([df.reset_index(), dumm.reset_index()], axis=1)
# df2.set_index(["sid", "aid", "wid", "mid"], inplace=True)
df2.index = [[x for x in range(e.EXAMPLES_LEN) for _ in range(e.SENTLEN)],
             [x for _ in range(e.EXAMPLES_LEN) for x in range(e.SENTLEN)]]

if len(['embeddings' + str(x) for x in range(e.EMBEDDINGS)]) > 0:
    df2.drop(['embeddings' + str(x) for x in range(e.EMBEDDINGS)], inplace=True, axis=1)
# df2.columns = [(x,"val") if isinstance(x,str) else x  for x in df2.columns]

In [154]:
df2.columns = [["vals" if isinstance(x, str) else x[0] for x in df2.columns],
               [x if isinstance(x, str) else x[1] for x in df2.columns]]

df2.sort_index(axis=1, inplace=True)

In [155]:
del df
del dumm

In [159]:
df2[("is_fourty","is_fourty")] = df2[("vals","sid")].str.startswith("fourty")

## Pre-process String Features

In [160]:
def emb_encode(x):
    return np.zeros(e.emb.vector_size) if re.sub("\\s*","",x)=="" or x == -1 else e.emb[x]

def convertStringToFeatX(x):
    df2[(x,"emb")] = df2[(x,x+"_undiac")].apply(emb_encode)
    return df2.join(pd.DataFrame(np.stack(df2[(x,"emb")].values),
                         index=df2.index, 
                         columns=[[x+"_emb" for i in range(emb.vector_size)],
                                  [x+"_emb_"+str(i) for i in range(emb.vector_size)]]))

# if not e.CHARACTER_BASED and e.MORPHEME_BASED:
#     df2 = convertStringToFeatX("word")
#     e.feat_x.append("word_emb")

In [169]:
# b. group them by morpheme and join with "+""
df_strings = pd.DataFrame({
    x: df2["vals", x].groupby(level=[0]).apply(joinMorphemesStrings)
    for x in strings
})

In [162]:
# c. pad joined morphemes
e.STRING_LENGTH = max([len(x) for k in strings for x in df_strings[k]])
for s in strings:
    df_strings[s] = df_strings[s].apply(padStringWithSpaces)

NameError: name 'df_strings' is not defined

In [164]:
# d. encode them in one hot encoding
charset = set("+").union(
    *[list(set("".join(df_strings[x] + "-"))) for x in strings])
ctable = CharacterTable(charset, e.STRING_LENGTH)
### Now we have one shape for all strings: (STRING_LENGTH, len(charset))
for x in strings:
    df_strings[x + "_onehot"] = df_strings[x].apply(ctable.encode)
df_strings['num'] = [x for x in range(len(df_strings))]
df_strings.set_index('num', append=True, inplace=True)

NameError: name 'df_strings' is not defined

In [ ]:
# e. remove diac
for x in strings:
    df_strings[x+ "_undiac"] = df_strings[x].apply(removeDiac)

In [ ]:
e.LOAD_FASTTEXT = False

In [ ]:
# f. encode them as dense vector using fastText
for x in e.strings_x:
    if e.LOAD_FASTTEXT:
        df_strings[x + "_emb"] = df_strings[x+"_undiac"].apply(emb_encode)

In [ ]:
def emb_encode_char(x):
    return ctable.encode(x)[0].argmax()
if e.CHARACTER_BASED:
    df2["bw_onehot"] = df2[("bw","bw")].astype(str).apply(emb_encode_char)
    df2.filter(regex="bw")

## 4.1 Align Input and Output Strings

In [ ]:
wutfs = df2.loc[(slice(None),0),("vals",e.strings_x[0])].tolist()
wutfs = [x.replace("\n","") for x in wutfs]

In [ ]:
utfs = df2.replace(-1,"")[df2[("vals","QAutf8")]!=""][("vals","QAutf8")].groupby(level=[0]).apply("+".join).map(lambda x: re.sub("\+*$","",x)).tolist()

In [ ]:
import edit_distance

In [ ]:
is_letter = "[^aFuNiKo~`۟@#\[^]"
is_diacritic = "[aFuNiKo~`۟@#\[^]"
utfs_grouped = [re.findall(is_letter+is_diacritic+"*",utf2bw(x)) for x in utfs]
wutfs_grouped = [re.findall(is_letter+is_diacritic+"*",utf2bw(x.replace("+",""))) for x in wutfs]

utfs_grouped_letters_noPlus = [re.findall(is_letter,utf2bw(x).replace("+","")) for x in utfs]
utfs_grouped_letters = [re.findall(is_letter,utf2bw(x)) for x in utfs]
wutfs_grouped_letters = [re.findall(is_letter,utf2bw(x.replace("+",""))) for x in wutfs]


In [ ]:
[i for i in utfs+wutfs if type(i)!=str]

In [ ]:
# import math
def getLetterBased(x,utfs_grouped_letters, wutfs_grouped_letters):
    sm = edit_distance.SequenceMatcher(utfs_grouped_letters[x],wutfs_grouped_letters[x])
#     sm2 = edit_distance.SequenceMatcher(utfs_grouped_letters_noPlus[x],wutfs_grouped_letters[x])
#     if sm2.ratio() == 1:
#         return
    a = []
    b = []
    c = []
    counter = 0
    atmp = []
    acounter = []
    for tag, i1, i2, j1, j2 in sm.get_opcodes():
        if tag=="delete" and utfs_grouped[x][i1] == "+":
            if len(c) > 0:
                c[-1] = [counter]
            counter+=1
            continue
        if tag == "replace":
            if utfs_grouped[x][i1] == "+":
                a.append(atmp) # means nothing
                b.append(wutfs_grouped[x][j1])
                c.append(acounter+[counter])                
                atmp = []
                acounter = []

                if len(c) > 0:
                    c[-1] = [counter]
                counter+=1
                continue
            else:
                tag = "equal"
        if tag=="equal":
            a.append(atmp+[utfs_grouped[x][i1]])
            b.append(wutfs_grouped[x][j1])
            c.append(acounter+[counter])                
            atmp = []
            acounter = []
        elif tag=="delete":
            atmp.append(utfs_grouped[x][i1])
            acounter.append(counter)
        elif tag=="insert":
            a.append([]) # means nothing
            b.append(wutfs_grouped[x][j1])
#     for cc in c:
#         for ccc in cc:
#             if type(df2.loc[(x,ccc),("vals","QApos")])!= str:
#                 print("Error at ", x, cc, df2.loc[(x,ccc),("vals","QApos")])
#                 print(df2.loc[(x,slice(None)),:])
            
    return [[x, "+".join([str(df2.loc[(x,ccc),("vals","QApos")]) for ccc in cc]), "+".join(a[i]), b[i],"".join([ str(ccc) for ccc in cc])] for i, cc in enumerate(c) ]

In [ ]:
letter_based = [getLetterBased(x, utfs_grouped_letters, wutfs_grouped_letters) for x in range(len(utfs_grouped_letters))]

In [ ]:
themax = max([len(x) for x in letter_based])

In [ ]:
letter_based2 =[]
for i in range(len(letter_based)):
    if letter_based[i] is not None:
        letter_based2.append(letter_based[i] + [[i,'-1','','',''] for xx in range(len(letter_based[i]), themax)])
#         letter_based2.append(letter_based[i])
        letter_based2[i] = [xx +[
                re.sub(is_diacritic+"*","",xx[2]), 
                re.sub(is_letter,"",xx[2]),
                re.sub(is_diacritic+"*","",xx[3]), 
                re.sub(is_letter,"",xx[3])
                ] for xx in letter_based2[i]]


In [ ]:
class CharacterTable2(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((len(self.chars),))
        x[self.char_indices[C]] = 1
        return x

    def getCode(self, C):
        return self.char_indices[C]

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return self.indices_char[x]


In [ ]:
df_chstrings = pd.DataFrame(np.array(letter_based2).reshape((-1,9)), 
                            columns= ["wid","pos","in","out","seg","in_letter","in_diac","out_letter","out_diac"],
                            index=[#[x for x in range(len(letter_based2)) for xx in range(themax)],
                                   list(range(1,themax+1))*len(letter_based2)])

In [ ]:
df_chstrings["wid"] = df_chstrings["wid"].astype(int)

In [ ]:
df_chstrings.set_index("wid", append=True, inplace=True, verify_integrity=True)

In [ ]:
ctable2s = {x: CharacterTable2(df_chstrings[x]) for x in e.strings_x + e.strings_y}
### Now we have one shape for all strings: (STRING_LENGTH, len(charset))
for x in e.strings_x + e.strings_y:
    print(x)
    df_chstrings[x + "_onehot"] = df_chstrings[x].apply(ctable2s[x].getCode)

In [ ]:
if "is_fourty" not in e.feat_x:
    e.feat_x.append("is_fourty")

In [ ]:
df2[("vals","sentid")] = df2[("vals","sid")] + df2[("vals","aid")].astype(str)

In [ ]:
df2.set_index([("vals","sentid"),("vals","wid"),"level_1"])

In [ ]:
df2.values.shape

# 6. Prepare splits

In [ ]:
e.SHUFFLE=False

In [ ]:
if not e.SHUFFLE:
    from collections import Counter

    ids = [str(x[0])+","+x[1] for x in df2.filter(regex="sid|aid").values.tolist()]
    ids_set = list(set(ids))
    cntr = Counter(ids)
    # cntr
    ids
    np.random.seed(0)
    np.random.shuffle(ids_set)
    total = 0
    val_split_at = 0
    test_split_at = 0

    for x in range(len(ids_set)):
        total += cntr[ids_set[x]]
        if total > len(df2)* e.TEST_SPLIT / 10 + len(df2)* e.VAL_SPLIT / 10 and val_split_at == 0:
            val_split_at = x
        if total > len(df2)* e.TEST_SPLIT / 10 and test_split_at == 0:
            test_split_at = x


In [ ]:
def getMorphemeBasedIndeicies(arr):
    return np.array([list(range(x*e.SENTLEN,x*e.SENTLEN+e.SENTLEN)) for x in arr]).flatten()

In [ ]:
if e.SHUFFLE:
    # 6. Shuffle (x, y) in unison
    np.random.seed(0)
    indices = list(range(e.EXAMPLES_LEN))
    np.random.shuffle(indices)
    # 7. Explicitly set apart 10% for validation data that we never train over.
    test_split_at = int(e.EXAMPLES_LEN * e.TEST_SPLIT / 10)
    val_split_at = int(e.EXAMPLES_LEN * e.TEST_SPLIT / 10 +
                       e.EXAMPLES_LEN * e.VAL_SPLIT / 10)

    values_test = df_strings.iloc[indices[:test_split_at]]
    values_val = df_strings.iloc[indices[test_split_at:val_split_at]]
    values_train = df_strings.iloc[indices[val_split_at:]]

    test = df2.iloc[getMorphemeBasedIndeicies(indices[:test_split_at])]
    val = df2.iloc[getMorphemeBasedIndeicies(indices[test_split_at:val_split_at])]
    train = df2.iloc[getMorphemeBasedIndeicies(indices[val_split_at:])]
else:
    test_indecies = df2.drop("level_0",axis=1).reset_index(level=0).set_index([("vals","sid"), ("vals","aid")]).loc[
        [tuple([x.split(",")[1],int(x.split(",")[0])]) for x in ids_set[:test_split_at]]
        ,:].loc[:,["level_0","level_1"]]
    val_indecies = df2.drop("level_0",axis=1).reset_index(level=0).set_index([("vals","sid"), ("vals","aid")]).loc[
        [tuple([x.split(",")[1],int(x.split(",")[0])]) for x in ids_set[test_split_at:val_split_at]]
        ,:].loc[:,["level_0","level_1"]]

    values_test = df_chstrings.loc[[(xx+1,x) for x in list(test_indecies.loc[test_indecies["level_1"]==0,"level_0"]) for xx in range(themax)],:]
    values_val = df_chstrings.loc[[(xx+1,x) for x in list(val_indecies.loc[val_indecies["level_1"]==0,"level_0"]) for xx in range(themax)],:]
    values_train = df_chstrings.loc[~df_chstrings.index.isin(values_test.index.tolist()+values_val.index.tolist())]

    
    test = df2.loc[list(map(tuple, test_indecies.values)),:]
    val = df2.loc[list(map(tuple, val_indecies.values)),:]
    train = df2.loc[~df2.index.isin(list(map(tuple, np.concatenate((test_indecies.values,val_indecies.values), axis=0))))]

In [ ]:
if False:
    import pyperclip
    pyperclip.copy("\n".join(sorted(list(set([x for x in train[("QAwutf",e.strings_x[0])].tolist() if type(x)==str])))))

In [ ]:
embeddingInputLists = {
    i: [i + "_" + str(x) for x in embeddingInputSets[i]]
    for i in embeddingInputSets
}

In [ ]:
e.doFinal = True

In [ ]:
def getValuesAndReshape(df, middle_dim, test=""):
    return df.values.reshape((df.shape[0] // middle_dim, middle_dim, -1))

# def setZero(df,zeros, from_val=-1, to_val=0.):
# #     dumm[(dumm.isin([0,-1])).all(axis=1)] = -1.
#     if zeros:
#         df[(df==from_val).all(axis=1)] = to_val
#     return df

def getData(cat_source, str_source, cats_feats=[], strs_feats=[], others_feats=[], embeddings=False, zeros= False):
    data = {
        **{i:
           getValuesAndReshape(cat_source[i].idxmax(axis=1).apply(lambda x: embeddingInputLists[i].index(x)),e.SENTLEN)
               if i in embeddingInputSets and len(embeddingInputSets[i]) > 10000+e.CATS_EMBEDDING #and i.replace("_emb","") not in e.strings_x
               else getValuesAndReshape(cat_source[i], e.SENTLEN,i) 
           for i in cats_feats},
        **{i: getValuesAndReshape(cat_source[i], e.SENTLEN,i) for i in others_feats},
        **{i:
           np.eye(len(ctable2s[i].chars))[str_source[i+"_onehot"].values.reshape((-1, themax))] if zeros
           else str_source[i+"_onehot"].values.reshape((-1, themax))
           for i in strs_feats},
#         **{i:np.stack(str_source[i+"_onehot"].values.reshape((-1, themax))) for i in strs_feats},
    }
    if embeddings:
        for i in strs_feats:
            if i+"_emb" not in data:
                data[i+"_emb"] = np.stack(str_source[i+"_emb"].values)
    if e.CHARACTER_BASED:
        data["bw_onehot"] = cat_source["bw_onehot"].values.reshape((-1, e.SENTLEN))
    if zeros:
        for i in cats_feats:
            data[i][np.where(data[i]==-1.)] = 0.
    if e.doFinal and zeros:
        data["sparseFinal"] = np.sum(np.concatenate([
            np.reshape(np.argmax(data[x],axis=-1)*data[x].shape[-1]**i,newshape=(-1,e.SENTLEN,1)) 
            for i,x in enumerate(cats_feats)],
            axis=-1),axis=-1,keepdims=True)
    return data


data = {
    'input': getData(train, values_train, cats_feats=e.feat_x, strs_feats=e.strings_x, others_feats=e.others_x, embeddings=e.LOAD_FASTTEXT),
    'output': getData(train, values_train, cats_feats=e.feat_y, strs_feats=e.strings_y, zeros=True),
    'val': (
        getData(val, values_val, cats_feats=e.feat_x, strs_feats=e.strings_x, others_feats=e.others_x, embeddings=e.LOAD_FASTTEXT),
        getData(val, values_val, cats_feats=e.feat_y, strs_feats=e.strings_y, zeros=True)
    ),
    'test': (
        getData(test, values_test, cats_feats=e.feat_x, strs_feats=e.strings_x, others_feats=e.others_x, embeddings=e.LOAD_FASTTEXT),
        getData(test, values_test, cats_feats=e.feat_y, strs_feats=e.strings_y, zeros=True)
    )

}

In [ ]:
[(x,data["output"][x].shape[-1]) for x in e.feat_y]

In [ ]:
data["val"] = tuple([data["val"][0],data["val"][1],
                     {i: (data["val"][1][i].argmax(axis=2) > 0).astype(int) for i in e.feat_y + e.strings_y if i in data["val"][1]},]
                   )

# Build Model

In [ ]:
e.ENCH = True

In [ ]:
def getEmbedding(input):
    name = input.name.split("_")[0]
    if name in e.feat_x and len(embeddingInputSets[name]) > e.CATS_EMBEDDING:
        return layers.Reshape((e.SENTLEN, -1))(layers.Embedding(len(embeddingInputSets[name]),2, input_length=e.SENTLEN)(input))
    else:
        return layers.Dropout(0.01)(input)


## End-To-End

In [ ]:
if not e.LOAD_MODEL and not e.ENCH and not e.MORPHEME_BASED and not e.CHARACTER_BASED:
    # print('Build model...')
    outputs = []
    inputs = []
    # For strings
    strings_input = layers.Input(shape=(e.STRING_LENGTH, len(charset)), name=e.strings_x[0])
    lstm_strings_encoder = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE,name="lstm_strings_encoder"))(strings_input)

    if e.LOAD_FASTTEXT:
        emb_input = layers.Input(shape=(e.emb.vector_size,), name=e.strings_x[0]+"_emb")
        inputs.append(emb_input)
    if len(e.feat_x) == 0:
        inputs.append(strings_input)
        concatenated = layers.Concatenate()([lstm_strings_encoder,emb_input])
    else:
        # For categoricals
        feat_x_cat = {name: [x for x in e.feat_x if x[:2] == name] for name in set([y[:2] for y in e.feat_x]) }
        rnns = []
        for i in feat_x_cat:
            inp = []
            for j in feat_x_cat[i]:
                inp.append(layers.Input(shape=(e.SENTLEN, data["input"][j].shape[2]), name=j))

            # main_input = layers.Concatenate()([layers.Dropout(0.01)(input) for input in inputs])
            input_list = [getEmbedding(input) for input in inp]

            main_input = layers.Concatenate()(input_list)
            lstm_out = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE), name="RNN_"+i)(main_input)
            rnns.append(lstm_out)
            inputs +=inp

        inputs.append(strings_input)

        # input_shape=(None, len(ctable_x.chars) + EMBEDDINGS)))
        # As the decoder e.RNN's input, repeatedly provide with the last hidden state of
        # e.RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
        # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
        if e.LOAD_FASTTEXT:
            concatenated = layers.Concatenate()(rnns+[lstm_strings_encoder,emb_input])
        else:
            concatenated = layers.Concatenate()(rnns+[lstm_strings_encoder])

    # For strings again
    repeat_strings_out = layers.RepeatVector(e.STRING_LENGTH)(concatenated)
    rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(repeat_strings_out)
    strings_output = layers.TimeDistributed(
          layers.Dense(
            len(charset), 
            activation="softmax"), name=e.strings_y[0])(rnn_out)
    outputs.append(strings_output)

    dropout_out = layers.Dropout(0.01)(concatenated)
    repeat_out = layers.RepeatVector(e.SENTLEN)(dropout_out)
    # The decoder e.RNN could be multiple layers stacked or a single layer.
    rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(repeat_out)
    for _ in range(e.LAYERS-1):
        # By setting return_sequences to True, return not only the last output but
        # all the outputs so far in the form of (num_samples, timesteps,
        # output_dim). This is necessary as TimeDistributed in the below expects
        # the first dimension to be the timesteps.
        rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(rnn_out)



    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.

    for i in e.feat_y:
        outputs.append(
          layers.TimeDistributed(
          layers.Dense(
            data["output"][i].shape[2], 
            activation="softmax"), name=i)(rnn_out))

In [ ]:
if not e.LOAD_MODEL and not e.ENCH and not e.MORPHEME_BASED and not e.CHARACTER_BASED:
    # print('Build model...')
    outputs = []
    inputs = []
    # For strings
    strings_input = layers.Input(shape=(e.STRING_LENGTH, len(charset)), name=e.strings_x[0])
    lstm_strings_encoder = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE,name="lstm_strings_encoder"))(strings_input)

    if e.LOAD_FASTTEXT:
        emb_input = layers.Input(shape=(e.emb.vector_size,), name=e.strings_x[0]+"_emb")
        inputs.append(emb_input)
    if len(e.feat_x) == 0:
        inputs.append(strings_input)
        concatenated = layers.Concatenate()([lstm_strings_encoder,emb_input])
    else:
        # For categoricals
        feat_x_cat = {name: [x for x in e.feat_x if x[:2] == name] for name in set([y[:2] for y in e.feat_x]) }
        rnns = []
        for i in feat_x_cat:
            inp = []
            for j in feat_x_cat[i]:
                inp.append(layers.Input(shape=(e.SENTLEN, data["input"][j].shape[2]), name=j))

            # main_input = layers.Concatenate()([layers.Dropout(0.01)(input) for input in inputs])
            input_list = [getEmbedding(input) for input in inp]

            main_input = layers.Concatenate()(input_list)
            lstm_out = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE), name="RNN_"+i)(main_input)
            rnns.append(lstm_out)
            inputs +=inp

        inputs.append(strings_input)

        # input_shape=(None, len(ctable_x.chars) + EMBEDDINGS)))
        # As the decoder e.RNN's input, repeatedly provide with the last hidden state of
        # e.RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
        # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
        if e.LOAD_FASTTEXT:
            concatenated = layers.Concatenate()(rnns+[lstm_strings_encoder,emb_input])
        else:
            concatenated = layers.Concatenate()(rnns+[lstm_strings_encoder])

    # For strings again
    repeat_strings_out = layers.RepeatVector(e.STRING_LENGTH)(concatenated)
    rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(repeat_strings_out)
    strings_output = layers.TimeDistributed(
          layers.Dense(
            len(charset), 
            activation="softmax"), name=e.strings_y[0])(rnn_out)
    outputs.append(strings_output)

    dropout_out = layers.Dropout(0.01)(concatenated)
    repeat_out = layers.RepeatVector(e.SENTLEN)(dropout_out)

    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.

    for i in e.feat_y:
        # The decoder e.RNN could be multiple layers stacked or a single layer.
        rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(repeat_out)
        for _ in range(e.LAYERS-1):
            # By setting return_sequences to True, return not only the last output but
            # all the outputs so far in the form of (num_samples, timesteps,
            # output_dim). This is necessary as TimeDistributed in the below expects
            # the first dimension to be the timesteps.
            rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(rnn_out)
        outputs.append(
          layers.TimeDistributed(
          layers.Dense(
            data["output"][i].shape[2], 
            activation="softmax"), name=i)(rnn_out))

## 8.2 Morpheme-Based  or 8.3 Character-Based 

In [ ]:
if not e.LOAD_MODEL and not e.ENCH and e.MORPHEME_BASED or e.CHARACTER_BASED:
    # print('Build model...')
    outputs = []
    inputs = []

    # For categoricals
    for i in e.feat_x:
        inputs.append(layers.Input(shape=(e.SENTLEN, data["input"][i].shape[2]), name=i))

    def getEmbeddingWithMasking(input):
        name = input.name.split("_")[0]
        if name in e.feat_x and len(embeddingInputSets[name]) > e.CATS_EMBEDDING:
            return layers.Masking(mask_value=0.)(layers.Reshape((e.SENTLEN, -1))(layers.Embedding(len(embeddingInputSets[name]),2, input_length=SENTLEN)(input)))
        else:
            return layers.Dropout(0.1)(layers.Masking(mask_value=0.)(input))

    main_input = layers.Concatenate()([getEmbedding(input) for input in inputs])

    lstm_out = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE, return_sequences=True))(main_input)
    if e.LOAD_FASTTEXT and e.MORPHEME_BASED:
        emb_input = layers.Input(shape=(e.emb.vector_size,), name=e.strings_x[0]+"_emb")
        inputs.append(emb_input)
        rnn_out = layers.Concatenate()([lstm_out,layers.RepeatVector(e.SENTLEN)(emb_input)])
    elif e.CHARACTER_BASED:
        emb_input = layers.Input(shape=(e.SENTLEN,), name="bw_onehot")
        inputs.append(emb_input)
        rnn_out = layers.Concatenate()([lstm_out,layers.Embedding(len(ctable.chars),10)(emb_input)])
    else:
        rnn_out = lstm_out





    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.

    for i in e.feat_y:
        outputs.append(
          layers.TimeDistributed(
          layers.Dense(
            data["output"][i].shape[2], 
            activation="softmax"), name=i)(rnn_out))

## 8.3 Character End-to-End 

In [ ]:
e.posfeat_y = "QApos"

In [ ]:
e.others_x = ["is_fourty"]

In [ ]:
tmp = [data["output"][x] for x in e.feat_y]
# tmp = [np.array([[[0,1],[1,0]]]),np.array([[[0,0,1],[0,0,1]]])]
# tmpo = np.array([[15,15]])
import tensorflow as tf
def MultipeOutputToSparseOne(x):
    if not isinstance(x, list) or len(x) < 2:
        raise ValueError('This layer should be called '
                         'on a list of at least 2 inputs')
#     print(x)
    return K.sum(K.reshape(K.concatenate([K.reshape(            
        K.cast(
        K.argmax(tensor, axis=-1) * int(tensor.get_shape()[-1])**ii
        , dtype="float32")
        , shape=(-1,x[0].get_shape()[1],1))          
                                          for ii, tensor in enumerate(x)]
                                                  ), 
                                     shape=(-1,x[0].get_shape()[1],len(x))),axis=-1, keepdims=True)

def sparse_cross_entropy(y_true, y_pred):
    """
    Calculate the cross-entropy loss between y_true and y_pred.
    
    y_true is a 2-rank tensor with the desired output.
    The shape is [batch_size, sequence_length] and it
    contains sequences of integer-tokens.

    y_pred is the decoder's output which is a 3-rank tensor
    with shape [batch_size, sequence_length, num_words]
    so that for each sequence in the batch there is a one-hot
    encoded array of length num_words.
    """

    # Calculate the loss. This outputs a
    # 2-rank tensor of shape [batch_size, sequence_length]
    tf.nn.sparse_softmax_cross_entropy_with_logits()
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,
                                                          logits=y_pred)

    # Keras may reduce this across the first axis (the batch)
    # but the semantics are unclear, so to be sure we use
    # the loss across the entire 2-rank tensor, we reduce it
    # to a single scalar with the mean function.
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

#                      num_classes=sum([int(tensor.get_shape()[-1])**ii for ii, tensor in enumerate(x)]))

In [ ]:
tmpinp = [layers.Input(shape=(e.SENTLEN,data["output"][x].shape[-1])) for x in e.feat_y]
# tmpinp = [layers.Input(shape=(2,2),dtype="int64"),layers.Input(shape=(2,3),dtype="int64")]
# print(layers.Lambda(MultipeOutputToSparseOne)(tmpinp))
tmpmodel = Model(inputs=tmpinp, outputs=#layers.Lambda(lambda x: K.cast(x, dtype="int32"))(
                                                          layers.Lambda(MultipeOutputToSparseOne, trainable=False, name="me")(tmpinp)
#                                                          , -1)
                                         #            )
                                        )

# decoder_target = tf.placeholder(dtype='int32', shape=(None, e.SENTLEN))

tmpmodel.compile(
              loss="mean_squared_error",#sparse_cross_entropy,#'sparse_categorical_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'],
#               target_tensors=[decoder_target]
            )
tmpmodel.predict(tmp)

In [ ]:
from keras import backend as K
from functools import reduce
if e.ENCH == True:
    # print('Build model...')
    outputs = []
    inputs = []
    strings_inputs = {}
    # For strings
    for i in e.strings_x:
        strings_inputs[i] = layers.Input(shape=(themax, ), name=i)
        inputs.append(strings_inputs[i])
    embedding_layers = [layers.Embedding(len(ctable2s[i].chars),10)(strings_inputs[i]) for i in strings_inputs]
    
#     lstm_strings_encoder = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE,name="lstm_strings_encoder"))(layers.Concatenate()(embedding_layers))

    lstm_strings_encoder2 = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE,name="lstm_strings_encoder", return_sequences=True))(layers.Concatenate()(embedding_layers))

    lstm_strings_encoder = layers.Lambda(lambda x: x[:, -1, :],name="LastState_returnSeqFalse")(lstm_strings_encoder2) # get the last state only


#     if e.LOAD_FASTTEXT:
#         emb_input = layers.Input(shape=(e.emb.vector_size,), name=e.strings_x[0]+"_emb")
#         inputs.append(emb_input)
    if len(e.feat_x) == 0:
#         inputs.append(strings_input)
        concatenated = layers.Concatenate()([lstm_strings_encoder,emb_input])
    else:
        # For categoricals
        feat_encoders = []
#   BLOCK1 : THIS IS makes an encoder to each GROUP: e.g. AM or gender. 
#         feat_x_cat = {name: [x for x in e.feat_x if x[:2] == name] for name in set([y[:2] for y in e.feat_x])}
#         for i in feat_x_cat:
#             inp = []
#             for j in feat_x_cat[i]:
#                 inp.append(layers.Input(shape=(e.SENTLEN, data["input"][j].shape[2]), name=j))

#             # main_input = layers.Concatenate()([layers.Dropout(0.01)(input) for input in inputs])
#             input_list = [getEmbedding(input) for input in inp]

#             main_input = layers.Concatenate(name="Conc_"+i)(input_list)
#             lstm_out = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE), name="RNN_"+i)(main_input)
#             feat_encoders.append(lstm_out)
#             inputs +=inp
#   BLOCK1 : ENDS HERE
        inp = [layers.Input(shape=(e.SENTLEN, data["input"][j].shape[2]), name=j) for j in e.feat_x]
        inputs +=inp
        inp = layers.Concatenate(name="Conc_Feat_X")([getEmbedding(input) for input in inp])
        feat_encoders.append(layers.Bidirectional(e.RNN(e.HIDDEN_SIZE * 4), name="RNN_All")(inp))
            
        non_tagger_features = [layers.Input(shape=(e.SENTLEN, data["input"][y].shape[2]), name=y) for y in e.others_x]
        inputs += non_tagger_features
        non_tagger_features = [layers.Lambda(lambda x: x[:,0,:], name="getFirstTimeStep")(i) for i in non_tagger_features]
#         inputs.append(strings_input)

        # input_shape=(None, len(ctable_x.chars) + EMBEDDINGS)))
        # As the decoder e.RNN's input, repeatedly provide with the last hidden state of
        # e.RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
        # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
        if e.LOAD_FASTTEXT:
            concatenated = layers.Concatenate(name="all_inputs")(feat_encoders+[lstm_strings_encoder,emb_input] + non_tagger_features)
        else:
            concatenated = layers.Concatenate(name="all_inputs")(feat_encoders+[lstm_strings_encoder] + non_tagger_features)

    
    # For strings again
#     repeat_strings_out = layers.RepeatVector(e.STRING_LENGTH)(concatenated)
#     rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(repeat_strings_out)
    for i in e.strings_y:
        strings_output = layers.TimeDistributed(
              layers.Dense(
                len(ctable2s[i].chars), 
                activation="softmax"), name=i)(lstm_strings_encoder2)
        outputs.append(strings_output)

    dropout_out = layers.Dropout(0.01)(concatenated)
    repeat_out = layers.RepeatVector(e.SENTLEN)(dropout_out)
    # The decoder e.RNN could be multiple layers stacked or a single layer.
    rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(repeat_out)
    for _ in range(e.LAYERS-1):
        # By setting return_sequences to True, return not only the last output but
        # all the outputs so far in the form of (num_samples, timesteps,
        # output_dim). This is necessary as TimeDistributed in the below expects
        # the first dimension to be the timesteps.
        rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(rnn_out)
        
    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.
    
    final_concat = []
    oo= layers.TimeDistributed(
      layers.Dense(
        data["output"][e.posfeat_y].shape[2], 
        activation="softmax"), name=e.posfeat_y)(rnn_out)
    final_concat.append(oo)
    outputs.append(oo)
    
    for i in e.feat_y:
        if i == e.posfeat_y:
            continue
        o= layers.TimeDistributed(
          layers.Dense(
            data["output"][i].shape[2], 
            activation="softmax"), name=i)(layers.Concatenate()([rnn_out,oo]))
        final_concat.append(o)
        outputs.append(o)
    outputs.append(layers.Lambda(MultipeOutputToSparseOne, name="sparseFinal")(final_concat))


In [ ]:
# THIS IS STATEFUL NETWORK.
# e.BATCH_SIZE = 1
# from keras import backend as K
# from functools import reduce
# if e.ENCH == True:
#     # print('Build model...')
#     outputs = []
#     inputs = []
#     strings_inputs = {}
#     # For strings
#     for i in e.strings_x:
#         strings_inputs[i] = layers.Input(batch_shape=(e.BATCH_SIZE,themax, ), name=i)
#         inputs.append(strings_inputs[i])
#     embedding_layers = [layers.Embedding(len(ctable2s[i].chars),10)(strings_inputs[i]) for i in strings_inputs]
    
# #     lstm_strings_encoder = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE,name="lstm_strings_encoder"))(layers.Concatenate()(embedding_layers))

#     lstm_strings_encoder2 = layers.Bidirectional(e.RNN(
#         e.HIDDEN_SIZE,name="lstm_strings_encoder", 
#         return_sequences=True,
#         stateful=True))(layers.Concatenate()(embedding_layers))

#     lstm_strings_encoder = layers.Lambda(lambda x: x[:, -1, :])(lstm_strings_encoder2) # get the last state only


# #     if e.LOAD_FASTTEXT:
# #         emb_input = layers.Input(shape=(e.emb.vector_size,), name=e.strings_x[0]+"_emb")
# #         inputs.append(emb_input)
#     if len(e.feat_x) == 0:
# #         inputs.append(strings_input)
#         concatenated = layers.Concatenate()([lstm_strings_encoder,emb_input])
#     else:
#         # For categoricals
#         feat_x_cat = {name: [x for x in e.feat_x if x[:2] == name] for name in set([y[:2] for y in e.feat_x]) if name != "is"}
#         rnns = []
#         for i in feat_x_cat:
#             inp = []
#             for j in feat_x_cat[i]:
#                 inp.append(layers.Input(batch_shape=(e.BATCH_SIZE, e.SENTLEN, data["input"][j].shape[2]), name=j))

#             # main_input = layers.Concatenate()([layers.Dropout(0.01)(input) for input in inputs])
#             input_list = [getEmbedding(input) for input in inp]

#             main_input = layers.Concatenate(name="Conc_"+i)(input_list)
#             lstm_out = layers.Bidirectional(e.RNN(e.HIDDEN_SIZE,stateful=True), name="RNN_"+i)(main_input)
#             rnns.append(lstm_out)
#             inputs +=inp
            
#         non_tagger_features = [layers.Input(batch_shape=(e.BATCH_SIZE,e.SENTLEN, data["input"][y].shape[2]), name=y) for y in e.feat_x if y[:2] == "is"]
#         inputs += non_tagger_features
#         non_tagger_features = [layers.Lambda(lambda x: x[:,0,:])(i) for i in non_tagger_features] #output_shape=(1,) + input_shape[2:])]
# #         inputs.append(strings_input)

#         # input_shape=(None, len(ctable_x.chars) + EMBEDDINGS)))
#         # As the decoder e.RNN's input, repeatedly provide with the last hidden state of
#         # e.RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
#         # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#         if e.LOAD_FASTTEXT:
#             concatenated = layers.Concatenate(name="all_inputs")(rnns+[lstm_strings_encoder,emb_input] + non_tagger_features)
#         else:
#             concatenated = layers.Concatenate(name="all_inputs")(rnns+[lstm_strings_encoder] + non_tagger_features)

    
#     # For strings again
# #     repeat_strings_out = layers.RepeatVector(e.STRING_LENGTH)(concatenated)
# #     rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(repeat_strings_out)
#     for i in e.strings_y:
#         strings_output = layers.TimeDistributed(
#               layers.Dense(
#                 len(ctable2s[i].chars), 
#                 activation="softmax"), name=i)(lstm_strings_encoder2)
#         outputs.append(strings_output)

#     dropout_out = layers.Dropout(0.01)(concatenated)
#     repeat_out = layers.RepeatVector(e.SENTLEN)(dropout_out)
#     # The decoder e.RNN could be multiple layers stacked or a single layer.
#     rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True,stateful=True)(repeat_out)
#     for _ in range(e.LAYERS-1):
#         # By setting return_sequences to True, return not only the last output but
#         # all the outputs so far in the form of (num_samples, timesteps,
#         # output_dim). This is necessary as TimeDistributed in the below expects
#         # the first dimension to be the timesteps.
#         rnn_out = e.RNN(e.HIDDEN_SIZE, return_sequences=True)(rnn_out)
        
#     # Apply a dense layer to the every temporal slice of an input. For each of step
#     # of the output sequence, decide which character should be chosen.
    
#     oo= layers.TimeDistributed(
#       layers.Dense(
#         data["output"][e.posfeat_y].shape[2], 
#         activation="softmax"), name=e.posfeat_y)(rnn_out)
#     outputs.append(oo)
    
#     for i in e.feat_y:
#         if i == e.posfeat_y:
#             continue
#         o= layers.TimeDistributed(
#           layers.Dense(
#             data["output"][i].shape[2], 
#             activation="softmax"), name=i)(layers.Concatenate()([rnn_out,oo]))
#         outputs.append(o)
# final = layers.Lambda(MultipeOutputToSparseOne)(final_concat)
# final                         

## Compile

In [ ]:
if not e.LOAD_MODEL:
    model = Model(inputs=inputs, outputs=outputs)

    model.compile(sample_weight_mode="temporal", #if not e.MORPHEME_BASED and not e.CHARACTER_BASED else None,
                  loss={x:'categorical_crossentropy' if x!="sparseFinal" else 'mean_squared_error' for x in model.output_names},
#                   loss_weights={x: 1.0 if x in ["QAutf8","QApos"] else 0.2 for x in model.output_names},
                  optimizer='adam',
                  metrics=['accuracy']
                )
else:
    from os.path import exists
    from os import listdir
    print("models/" + e.NAME + "_{}.keras".format(e.thedate))
    if exists("models/" + e.NAME + "_{}.keras".format(e.thedate)):
        model = load_model("models/" + e.NAME + "_{}.keras".format(e.thedate))
    else:
        print("possible models: (please change the date to reload model)")
        print([f for f in listdir("models/") if f.startswith(e.NAME)])
        d = input("the new date? (empty to keep the original one)")
        if d != "":
            e.thedate = "."+d
            print("date has been changed. Please re-run this code")
        else:
            print("date has not been changed.")

# 9. Summary

In [ ]:
model.summary()

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
if not e.LOAD_MODEL:
    plot_model(model, to_file='plots/model_{}.png'.format(e.NAME), )#show_shapes=True)
SVG(model_to_dot(model, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))

# 10. Training

In [ ]:
# optional
e.set_name()

In [ ]:
if not e.LOAD_MODEL:
    earlyStopping=EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

    tensorboard = TensorBoard(
        log_dir="logs/" + e.NAME + "_{}".format(e.thedate))


In [ ]:
max_len = 20
class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0
        self.wid_endings = [x[0] for x in list(train[train[("vals",e.posfeat_y)]==0].index)]

    def on_batch_begin(self, batch, logs={}):
        if self.counter in self.wid_endings:
            self.model.reset_states()
        self.counter += 1

In [ ]:
print(e.NAME + "_{}".format(e.thedate))


In [ ]:
if not e.LOAD_MODEL:
    history = model.fit(data['input'], data['output'],
                    batch_size=e.BATCH_SIZE,
                    shuffle=False,
                    callbacks=[
#                         ResetStatesCallback(), 
                        earlyStopping, 
#                                TestCallback(data['test']), 
                               tensorboard],
                    epochs=e.EPOCHS,
                    verbose=2,
                    sample_weight={i:((data["output"][i].argmax(axis=2) > 0).astype(int)*.9)+.1 for i in model.output_names},
                    validation_data=data['val'])

In [ ]:
if not e.LOAD_MODEL:
    model.save("models/" + e.NAME + "_{}.keras".format(e.thedate))

## 11. Evaluate Trained Model

In [ ]:
reduced_mapping = {"N": "NOUN","PN": "PROPN","ADJ": "ADJ","IMPN": "ADJ","PRP": "PRON","DEM": "PRON","REL": "PRON","PRON": "PRON","LOC": "ADV","T": "ADV","V": "VERB","P": "ADP","CONJ": "CCONJ","SUB": "SCONJ","ACC": "PART","AMD": "PART","ANS": "PART","AVR": "PART","CAUS": "PART","CERT": "PART","CIRC": "PART","COM": "PART","COND": "PART","EQ": "PART","EXH": "PART","EXL": "PART","EXP": "PART","FUT": "PART","INC": "PART","INT": "PART","INTG": "PART","NEG": "PART","PREV": "PART","PRO": "PART","REM": "PART","RES": "PART","RET": "PART","RSLT": "PART","SUP": "PART","SUR": "PART","VOC": "PART","INL": "PART","EMPH": "PART","IMPV": "PART","PRP": "PART","DET": "DET","INTJ": "INTJ","X": "X","SYM": "SYM","PUNC": "PUNCT","NUM": "NUM", "-1": "-1", "_": "_", "-": "-","0":"0"}

In [ ]:
def calc_accuracy(model,mydata,data_length, debug=True,train_vocab=[], train_input_variable=""):
    preds = [np.argmax(x, axis=-1) for x in model.predict(mydata[0])]
    results = pd.DataFrame([], columns=[["inputs"]*(len(e.strings_x)+2)+(e.strings_y+e.feat_y)*3#+["agg"]*3+["QAupos"]*3
                                        ,[*e.strings_x,"index","OOV"]
                                        +["acc"]*len(e.strings_y+e.feat_y)
                                        +["pred"]*len(e.strings_y+e.feat_y)
                                        +["actu"]*len(e.strings_y+e.feat_y)
                                        #+["all","utf8","mf"]
                                        #+ ["acc","pred","actu"]
                                       ])
    results.sort_index(axis=1, inplace=True)
    for ite in range(data_length):
        df2_row = mydata[2].iloc[ite*5]
        r = {
            ("inputs","index"): str(df2_row[("vals","sid")])+"-"+str(df2_row[("vals","aid")])+"-"+str(df2_row[("vals","wid")])+"-"+str(df2_row[("vals","mid")])
        }
        for i in e.strings_x:
            r[("inputs",i)] = "".join(ctable2s[i].decode(np.eye(len(ctable2s[i].chars))[ii], calc_argmax=True) for ii in mydata[0][i][ite])
        
        r[("inputs","OOV")] = r[("inputs",train_input_variable)] not in train_vocab if train_input_variable != "" else False
        for i, v in enumerate(model.output_names):
            if v not in e.strings_y:
                continue
            r[(v,"acc")] = (np.argmax(mydata[1][v][ite], axis=-1) == preds[i][ite]).all()
            r[(v,"pred")] = utf2bw("".join(ctable2s[v].decode(np.eye(len(ctable2s[v].chars))[ii]) for ii in preds[i][ite])).strip(" ") if debug else ""
            r[(v,"actu")] = utf2bw("".join(ctable2s[v].decode(ii, calc_argmax=True) for ii in mydata[1][v][ite])).strip(" ") if debug else ""

        for i, v in enumerate(model.output_names):
            if v not in e.feat_y:
                continue
            correct, pred = np.argmax(mydata[1][v][ite], axis=-1), preds[i][ite]
            r[(v,"acc")] = (correct == pred).all()
            r[(v,"actu")] = pretty_join2(correct,val[v].columns,v) if debug else ""
            r[(v,"pred")] = pretty_join2(pred,val[v].columns,v) if debug else ""
#         r[("agg","all")] = sum([r[(v,"acc")] for i, v in enumerate(model.output_names)]) == len(model.output_names)
#         r[("agg","utf8")] = sum([r[(v,"acc")] for i, v in enumerate(model.output_names) if v != "QAutf8" ]) == len([x for i,v in enumerate(model.output_names) if v != "QAutf8"])
#         r[("agg","mf")] = sum([r[(v,"acc")] for i, v in enumerate(model.output_names)  if v != "QAutf8" and v != "QApos" ]) == len([x for i,v in enumerate(model.output_names) if v != "QApos"])
        results.loc[ite] = r
    return results

In [ ]:
train_input_variable = "in_letter"
# train_vocab=set([x for x in train[("QAwutf","QAwutf8")].tolist() if type(x)==str])
train_vocab = ["".join(ctable2s[train_input_variable].decode(np.eye(len(ctable2s[train_input_variable].chars))[ii], 
                                                             calc_argmax=True) for ii in ite) 
               for ite in data["input"][train_input_variable]]
result = calc_accuracy(model, [data["val"][0],data["val"][1],val], len(data["val"][0][e.strings_x[0]]),debug=True,
                       train_vocab=train_vocab,
                       train_input_variable=train_input_variable
                      )
# result = calc_accuracy(model, [*data["test"],test], len(data["test"][0][e.strings_x[0]]),debug=True, 
#                        train_vocab=train_vocab, 
#                        train_input_variable=train_input_variable)

In [ ]:
def getUPosTag(xx, reduced_mapping=None):
    return "/".join([reduced_mapping[x] for x in xx.split("/")]) if reduced_mapping is not None else ""

result[("QAupos","actu")] = result[("QApos","actu")].apply(getUPosTag, args=(reduced_mapping,))
result[("QAupos","pred")] = result[("QApos","pred")].apply(getUPosTag, args=(reduced_mapping,))
result[("QAupos","acc")] = result[("QAupos","actu")] == result[("QAupos","pred")]

result[("inputs","ar")] = result[("inputs","in_letter")].apply(bw2utf)

In [ ]:
def printAcc(df, regex):
    print("Columns:", end="\t")
    print("\t".join([x[0] for x in df.filter(regex=regex).columns]))

    print("Agg Accuracy:", end="\t")
    rr = df
    print(len(rr[rr.filter(regex=regex).astype(int).sum(axis=1) == len(rr.filter(regex=regex).columns)])/len(rr))

    print("OOV Accuracy:", end="\t")
    rr = df[df["inputs","OOV"]==True]
    print(len(rr[rr.filter(regex=regex).astype(int).sum(axis=1) == len(rr.filter(regex=regex).columns)])/len(rr))

    print("IOV Accuracy:", end="\t")
    rr = df[df["inputs","OOV"]==False]
    print(len(rr[rr.filter(regex=regex).astype(int).sum(axis=1) == len(rr.filter(regex=regex).columns)])/len(rr))

In [ ]:
print("tagging")
printAcc(result,"QAaspect.*acc|QAcase.*acc|QAgender.*acc|QAmood.*acc|QAnumber.*acc|QAperson.*acc|QAstate.*acc|QAvoice.*acc|QApos.*acc")
print("tagging except gender and person")
printAcc(result,"QAaspect.*acc|QAcase.*acc|QAmood.*acc|QAperson.*acc|QAstate.*acc|QAvoice.*acc|QApos.*acc")
print("tagging except gender and person and pos")
printAcc(result,"QAaspect.*acc|QAcase.*acc|QAmood.*acc|QAperson.*acc|QAstate.*acc|QAvoice.*acc")
print("segmentation")
printAcc(result,"out.*acc|seg.*acc")
print("taggin and segmentation")
printAcc(result,"acc")

In [ ]:
def ShowWhereFalse(df,regex):
    return df[df.filter(regex=regex).astype(int).sum(axis=1) != len(df.filter(regex=regex).columns)].filter(regex="acc|in").replace(True,"")
ShowWhereFalse(result,regex)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(list(len(result.filter(regex="acc").columns) - result.filter(regex="acc").astype(int).sum(axis=1)), bins='auto')
plt.show()

In [ ]:
result.replace("(/-1)+", "", regex=True).replace(True, "").to_excel(e.writer,'results_test')
e.writer.save()

In [ ]:
try:
    results
except NameError:
    results = {}

if e.NAME not in results:
    results[e.NAME] = {}

In [ ]:
results[e.NAME]["results_val"]=result

In [ ]:
# del accuracies
try:
    accuracies
except NameError:
    accuracies = pd.DataFrame([],
                 columns=list(map(list, zip(*[
        ('QAaspect', 'acc'),('QAaspect', 'IOV_acc'),('QAaspect', 'OOV_acc'),
        ('QAcase', 'acc'),('QAcase', 'IOV_acc'),('QAcase', 'OOV_acc'),
        ('QAgender', 'acc'),('QAgender', 'IOV_acc'),('QAgender', 'OOV_acc'),
        ('QAmood', 'acc'),('QAmood', 'IOV_acc'),('QAmood', 'OOV_acc'),
        ('QAnumber', 'acc'),('QAnumber', 'IOV_acc'),('QAnumber', 'OOV_acc'),
        ('QAperson', 'acc'),('QAperson', 'IOV_acc'),('QAperson', 'OOV_acc'),
        ('QApos', 'acc'),('QApos', 'IOV_acc'),('QApos', 'OOV_acc'),
        ('QAupos', 'acc'),('QAupos', 'IOV_acc'),('QAupos', 'OOV_acc'),                     
        ('QAstate', 'acc'),('QAstate', 'IOV_acc'),('QAstate', 'OOV_acc'),
        ('QAvoice', 'acc'),('QAvoice', 'IOV_acc'),('QAvoice', 'OOV_acc'),
        ('out_letter', 'acc'),('out_letter', 'IOV_acc'),('out_letter', 'OOV_acc'),
        ('out_diac', 'acc'),('out_diac', 'IOV_acc'),('out_diac', 'OOV_acc'),
        ('seg', 'acc'),('seg', 'IOV_acc'),('seg', 'OOV_acc'),
        ('agg', 'all'),
        ('agg', 'mf'),
        ('agg', 'utf8')]))))
try:
    h_accuracies
except NameError:
    h_accuracies = pd.DataFrame([],
                 columns=list(map(list, zip(*[
        ('QAaspect', 'acc'),('QAaspect', 'IOV_acc'),('QAaspect', 'OOV_acc'),
        ('QAcase', 'acc'),('QAcase', 'IOV_acc'),('QAcase', 'OOV_acc'),
        ('QAgender', 'acc'),('QAgender', 'IOV_acc'),('QAgender', 'OOV_acc'),
        ('QAmood', 'acc'),('QAmood', 'IOV_acc'),('QAmood', 'OOV_acc'),
        ('QAnumber', 'acc'),('QAnumber', 'IOV_acc'),('QAnumber', 'OOV_acc'),
        ('QAperson', 'acc'),('QAperson', 'IOV_acc'),('QAperson', 'OOV_acc'),
        ('QApos', 'acc'),('QApos', 'IOV_acc'),('QApos', 'OOV_acc'),
        ('QAupos', 'acc'),('QAupos', 'IOV_acc'),('QAupos', 'OOV_acc'),                     
        ('QAstate', 'acc'),('QAstate', 'IOV_acc'),('QAstate', 'OOV_acc'),
        ('QAvoice', 'acc'),('QAvoice', 'IOV_acc'),('QAvoice', 'OOV_acc'),
#         ('QAutf8', 'acc'),('QAutf8', 'IOV_acc'),('QAutf8', 'OOV_acc'),
        ('out_letter', 'acc'),('out_letter', 'IOV_acc'),('out_letter', 'OOV_acc'),
        ('out_diac', 'acc'),('out_diac', 'IOV_acc'),('out_diac', 'OOV_acc'),
        ('seg', 'acc'),('seg', 'IOV_acc'),('seg', 'OOV_acc'),

        ('agg', 'all'),
        ('agg', 'mf'),
        ('agg', 'utf8')]))))

In [ ]:
r = {}
for x in list(accuracies.columns):
    if x in result.columns:
        r[x] = np.average(result[x].values.astype(int),axis=0)
    elif x[1] =="IOV_acc":
        r[x] = np.average(result[result[("inputs","OOV")]==False][(x[0],"acc")].values.astype(int),axis=0)
    elif x[1] =="OOV_acc":
        r[x] = np.average(result[result[("inputs","OOV")]==True][(x[0],"acc")].values.astype(int),axis=0)
    else:
        print(x)


r[('agg', 'all')] = len(result[result.filter(regex="acc").astype(int).sum(axis=1) == len(result.filter(regex="acc").columns)])/len(result)
r[('agg', 'mf')] = len(result[result.filter(regex="QAaspect.*acc|QAcase.*acc|QAgender.*acc|QAmood.*acc|QAnumber.*acc|QAperson.*acc|QAstate.*acc|QAvoice.*acc").astype(int).sum(axis=1) == len(result.filter(regex="QAaspect.*acc|QAcase.*acc|QAgender.*acc|QAmood.*acc|QAnumber.*acc|QAperson.*acc|QAstate.*acc|QAvoice.*acc").columns)])/len(result)
r[('agg', 'utf8')] = len(result[result.filter(regex="QA.*acc").astype(int).sum(axis=1) == len(result.filter(regex="QA.*acc").columns)])/len(result)

accuracies.loc[e.NAME+"_test"] = r
accuracies


In [ ]:
result_hadith = result[result[("inputs","index")].str.startswith("fourty")]
result_hadith.replace("(/-1)+", "", regex=True).replace(True, "").to_excel(e.writer,'results_hadith_test')
e.writer.save()

In [ ]:
r = {}
for x in list(h_accuracies.columns):
    if x in result_hadith.columns:
        r[x] = np.average(result_hadith[x].values.astype(int),axis=0)
    elif x[1] =="IOV_acc":
        r[x] = np.average(result_hadith[result_hadith[("inputs","OOV")]==False][(x[0],"acc")].values.astype(int),axis=0)
    elif x[1] =="OOV_acc":
        r[x] = np.average(result_hadith[result_hadith[("inputs","OOV")]==True][(x[0],"acc")].values.astype(int),axis=0)
    else:
        print(x)


h_accuracies.loc[e.NAME+"_test"] = r
h_accuracies

In [ ]:
result = calc_accuracy(model, [data["val"][0],data["val"][1],val], len(data["val"][0][e.strings_x[0]]),debug=True,
                       train_vocab=set([x for x in train[("QAwutf",e.strings_x[0])].tolist() if type(x)==str]), reduced_mapping=reduced_mapping)

In [ ]:
result.replace("(/-1)+", "", regex=True).replace(True, "").to_excel(e.writer,'results_val')
e.writer.save()

In [ ]:
result_hadith = result[result[("inputs","index")].str.startswith("fourty")]
result_hadith.replace("(/-1)+", "", regex=True).replace(True, "").to_excel(e.writer,'results_hadith_val')
e.writer.save()

In [ ]:
results[e.NAME]["results_val"]=result

In [ ]:
r = {x: np.average(result[x].values.astype(int),axis=0) 
     if x in result.columns else 0 for x in list(accuracies.columns)}

accuracies.loc[e.NAME+"_val"] = r
accuracies


In [ ]:
accuracies.to_excel(writer_all,'acc')
writer_all.save()

## 12. Charts

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.plot(history.history['test_loss'])
plt.title('model overall loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val', 'test'], loc='upper right')
plt.savefig('plots/model_overall_loss' +
            e.NAME + "_{}".format(e.thedate) + '.png')
plt.close(fig)

# plt.show()


# In[471]:

# summarize history for loss
plt.plot(history.history["train_acc"])
plt.plot(history.history['val_acc'])
plt.plot(history.history['test_acc'])
plt.title('model average accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend([x +" = "+ str(round(history.history[x+'_acc'][-1]*100,2)) for x in ['train', 'val', 'test']], loc='lower right')
# for i, x in enumerate(['train', 'val', 'test']):
#     plt.annotate(str(round(history.history[x+'_acc'][-1]*100,2)),
#                  xy=(len(history.history[x+'_acc']), history.history[x+'_acc'][-1]), 
#                  textcoords='figure pixels', 
#                  xytext=(-20,-10))
plt.savefig('plots/model_average_accuracy' +
            e.NAME + "_{}".format(e.thedate) + '.png')


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# summarize history for loss
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.plot(history.history['test_loss'])
plt.title('model overall loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val', 'test'], loc='upper right')
plt.savefig('plots/model_overall_loss' +
            e.NAME + "_{}".format(e.thedate) + '.png')

# plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# summarize history for loss
plt.plot(history.history["train_acc"])
plt.plot(history.history['val_acc'])
plt.plot(history.history['test_acc'])
plt.title('model average accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend([x +" = "+ str(round(history.history[x+'_acc'][-1]*100,2)) for x in ['train', 'val', 'test']], loc='lower right')
# for i, x in enumerate(['train', 'val', 'test']):
#     plt.annotate(str(round(history.history[x+'_acc'][-1]*100,2)),
#                  xy=(len(history.history[x+'_acc']), history.history[x+'_acc'][-1]), 
#                  textcoords='figure pixels', 
#                  xytext=(-20,-10))
plt.savefig('plots/model_average_accuracy' +
            e.NAME + "_{}".format(e.thedate) + '.png')

plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# VALIDATAION = True
# prefix = "val_" if VALIDATAION else ""
# del prefix
legends = []
for x in model.output_names:
    # summarize history for accuracy
    plt.plot(history.history[x+"_acc"])
    legends.append(""+x +" = "+ str(round(history.history[x+'_acc'][-1]*100,2)))
#     plt.plot(history.history[x+"_acc"])
#     legends.append("val_"+x)
#     plt.plot(history.history["val_" + x+"_acc"])
#     legends.append("train_"+x)
plt.title('model indiviual accuracy on test dataset')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(legends, loc='lower right')
plt.savefig('plots/accuracy_all' +
            e.NAME + "_{}".format(e.thedate) + '.png')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

legends = []
for x in model.output_names:
    # summarize history for loss
    plt.plot(history.history["test_"+x+"_loss"])
    legends.append(""+x +" = "+ str(round(history.history[x+'_loss'][-1]*100,2)))
plt.title('model individual loss on test dataset')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(legends, loc='upper right')
plt.savefig('plots/loss_all' +
            e.NAME + "_{}".format(e.thedate) + '.png')
plt.show()

## 10.1 Inspect One

In [ ]:
#%%capture --no-stderr cap
%autoreload 2

colors.ok = ''
colors.fail = ''
colors.close = ''


def inspectOne(times=0, printCorrect=True):
    isAllCorrect = True
    if times > 10000: # to prevent infinite loops
        return
    ind = np.random.randint(0, len(val.index))
    ind = (val.index[ind][0], slice(None))

    string_input = values_test.loc[(slice(None), ind[0]), :]
    preds = model.predict(
        getData(val.loc[ind], string_input, e.feat_x, e.strings_x))
    preds = [np.argmax(x, axis=-1) for x in preds]
    #predicted string
    print("Predicted String Q", string_input[e.strings_x[0]][0], "from",
          "-".join(str(x) for x in string_input.index.values[0]))

    if (np.argmax(string_input[e.strings_y[0]][0], axis=-1) == preds[0]).all():
        if printCorrect: print(colors.ok + '✅' + colors.close + "Segmentation")
    else:
        print(colors.fail + '❌' + colors.close + "Segmentation")
        isAllCorrect = False
        print("    T", utf2bw(string_input[e.strings_y[0]][0]))
        print("     ", utf2bw(ctable.decode(preds[0][0], calc_argmax=False)))


#     print('Q', utf2bw(pretty_join(rowx)))

    rowy = dict()
    for i, v in enumerate(e.feat_y):
        rowy[v] = {"correct": val[v].loc[ind]}
        res = np.zeros((SENTLEN, rowy[v]["correct"].shape[1]))
        for ii, c in enumerate(preds[i + 1][0]):
            res[ii, c] = 1
        rowy[v]["pred"] = pd.DataFrame(res, columns=val[v].columns)
        results = []
        if (rowy[v]["correct"].values == rowy[v]["pred"].values).all():
            if printCorrect: print(colors.ok + '✅' + colors.close + v)
        else:
            isAllCorrect = False
            print(colors.fail + '❌' + colors.close + v, end=' ')
            #             results.append(colors.fail + '☒' + colors.close)
            results.append('T ' + pretty_join(rowy[v]["correct"]))
            results.append(pretty_join(rowy[v]["pred"]))
            print(' '.join(results))
    if isAllCorrect or times < 10:
        print("")
        inspectOne(times + 1, printCorrect)

inspectOne(printCorrect=False)

with open(
        'output' + datetime.datetime.now().strftime(".%Y.%m.%d.%H.%M.%S") +
        '.txt', 'w') as f:
    f.write(cap.stdout)


### After alignment. Accuracy is good. Can be treated as baseline. (name=baseline)
`strings_cats_aligned_2018.06.25.15.14`
### Comaprison between baseline and POS embeddings. (No drop, so it is the best) (name=baseline+pos_emb)
`with_pos_embeddings_2018.06.25.17.37`
### Comaprison between baseline with POS embeddings and subword embeddings. (name=baseline+pos_emb+subword_emb)
`with_embeddings_2018.06.25.17.37` NOT DONE
### Comaprison between baseline with subword embeddings. (name=baseline+pos_emb+subword_emb+word2vec)
`with_word2vec_2018.06.25.17.37` NOT DONE

### Different Sizes of Baseline or subword